In [ ]:
import numpy as np
from IPython.display import clear_output
from matplotlib import pyplot as plt

from art import config
from art.estimators.classification import BlackBoxClassifier
from art.attacks.evasion import CarliniL2Method, SaliencyMapMethod
from art.utils import to_categorical
from art.utils import load_dataset, get_file, compute_accuracy